### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import multiprocessing as mp

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [2]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Dynamically Re-Load all Package Modules on Execution

In [3]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

#### Set File Path Constants

In [4]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [5]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Load Items Data

In [6]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Load Ratings Data

In [8]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

In [9]:
ratings_explicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,196,242,3,1997-12-04 15:55:49,0
1,186,302,3,1998-04-04 19:22:22,0
2,22,377,1,1997-11-07 07:18:36,0
3,244,51,2,1997-11-27 05:02:03,0
4,166,346,1,1998-02-02 05:33:16,0


#### Generate Implicit Feedback Ratings Data

In [10]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [11]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create (Training, Validation) Interactions for Evaluation

In [12]:
print(ratings_implicit.shape)
ratings_implicit.head()

(54194, 5)


,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


In [13]:
test_pct = 0.25
drop_users = (2, 4)

train_mask = (ratings_implicit.index / ratings_implicit.shape[0]) <  (1 - test_pct)
valid_mask = (ratings_implicit.index / ratings_implicit.shape[0]) >= (1 - test_pct)

interactions_train = ratings_implicit[train_mask][['user_id', 'item_id']]
interactions_valid = ratings_implicit[valid_mask][['user_id', 'item_id']]

# drop some users from the training data to test cold-start functionality
interactions_train = interactions_train[~interactions_train.user_id.isin(drop_users)]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))
print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

train shape: (40598, 2)
valid shape: (13548, 2)
train users: 941
valid users: 905
cold-start users: {2, 4}


### Test Out Core Package Functionality

#### Initialize the Model

In [82]:
model = RankFM(factors=10, learning_rate=0.1, regularization=0.01, sigma=0.1)
model

#### Fit the Model on the Training Data and Profile Computational Performance

In [83]:
%%prun -l 10 

model.fit(interactions_train, epochs=10, verbose=True)


training epoch: 0
penalized log-likelihood: -15731.57

training epoch: 1
penalized log-likelihood: -14738.4

training epoch: 2
penalized log-likelihood: -14099.35

training epoch: 3
penalized log-likelihood: -13350.38

training epoch: 4
penalized log-likelihood: -12233.47

training epoch: 5
penalized log-likelihood: -10842.86

training epoch: 6
penalized log-likelihood: -9980.11

training epoch: 7
penalized log-likelihood: -8932.02

training epoch: 8
penalized log-likelihood: -8445.46

training epoch: 9
penalized log-likelihood: -8058.86
 

         4383249 function calls (4381993 primitive calls) in 29.635 seconds

   Ordered by: internal time
   List reduced from 634 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1332   14.384    0.011   14.384    0.011 {method 'acquire' of '_thread.lock' objects}
   405980    9.355    0.000   10.998    0.000 rankfm.py:144(_gradient_step)
   811960    1.914    0.000    3.147    0.000 rankfm.py:132(_pairwise_utility)
   405990    1.529    0.000    3.033    0.000 rankfm.py:181(<genexpr>)
   812020    0.782    0.000    0.796    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   811960    0.374    0.000    1.233    0.000 <__array_function__ internals>:2(dot)
        1    0.303    0.303   29.631   29.631 rankfm.py:191(fit)
      360    0.181    0.001    0.181    0.001 {built-in method posix.waitpid}
       10    0.149    0.015    0.152    0.015 pool.py:375(_map_async)
       30    0.106    0.004    3.1

#### Evaluate Model Performance on the Validation Data

In [84]:
interactions_valid.head()

,user_id,item_id
40646,437,443
40647,712,1037
40648,903,1070
40649,151,629
40650,711,79


In [85]:
k = 10

In [86]:
# %%prun -l 10 

model_pre = round(precision_at_k(model, interactions_valid, k=k), 3)
model_rec = round(recall_at_k(model, interactions_valid, k=k), 3)
print("model precision: {} model recall: {}".format(model_pre, model_rec))

model precision: 0.118 model recall: 0.132


#### Evaluate Pure Popularity Baseline

In [68]:
most_popular = interactions_train.groupby('item_id').size().sort_values(ascending=False)[:k]
most_popular

item_id
50     357
100    291
181    277
258    260
98     249
174    246
127    245
1      225
56     223
313    221
dtype: int64

In [69]:
test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_recommends = model.recommend_for_users(list(test_user_items.keys()), cold_start='drop')
test_user_items = {key: val for key, val in test_user_items.items() if key in test_recommends.index}

base_pre = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()]), 3)
base_rec = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()]), 3)

print("number of test users: {}".format(len(test_user_items)))
print("baseline precision: {} baseline recall: {}".format(base_pre, base_rec))

number of test users: 903
baseline precision: 0.098 baseline recall: 0.088


#### Generate Model Scores for Test User/Items

In [70]:
mask = (interactions_valid.user_id.isin(interactions_train.user_id)) & (interactions_valid.item_id.isin(interactions_train.item_id))
print(interactions_valid.shape)
print(interactions_valid[mask].shape)

(13548, 2)
(13469, 2)


In [71]:
scores_miss = model.predict(interactions_valid, cold_start='nan')
scores_drop = model.predict(interactions_valid, cold_start='drop')

print(scores_miss.shape)
print(scores_drop.shape)

(13548,)
(13469,)


In [72]:
print(scores_miss.mean(), scores_miss.std(), scores_miss.min(), scores_miss.max())
print(scores_drop.mean(), scores_drop.std(), scores_drop.min(), scores_drop.max())

nan nan nan nan
1.5208763292916347 1.102956950618116 -3.3734460697931885 4.582640715335138


#### Spot-Check Some User Recommendations

In [73]:
len(train_users), len(valid_users), cold_start_users

(941, 905, {2, 4})

In [74]:
recommendations = model.recommend_for_users(valid_users, n_items=10, filter_previous=True, cold_start='nan')
print(recommendations.shape)
recommendations.head(10)

(905, 10)


,0,1,2,3,4,5,6,7,8,9
1,50.0,1.0,100.0,174.0,475.0,408.0,56.0,273.0,169.0,181.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,333.0,313.0,302.0,346.0,271.0,292.0,258.0,331.0,750.0,300.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,403.0,173.0,168.0,184.0,210.0,174.0,195.0,91.0,186.0,89.0
6,427.0,191.0,174.0,197.0,357.0,483.0,498.0,134.0,199.0,479.0
7,519.0,659.0,480.0,603.0,443.0,174.0,177.0,432.0,185.0,663.0
8,174.0,50.0,181.0,1.0,183.0,172.0,96.0,195.0,79.0,144.0
9,302.0,286.0,310.0,100.0,269.0,50.0,127.0,258.0,124.0,271.0
10,357.0,174.0,603.0,191.0,427.0,197.0,474.0,480.0,134.0,528.0


In [75]:
recommendations[recommendations.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
user_id = 6
user_recs = recommendations.loc[user_id]
user_item_names = item_names[item_names.item_id.isin(user_recs)].set_index('item_id').loc[user_recs]
user_item_names

,item_name
item_id,
427.0,To Kill a Mockingbird (1962)
191.0,Amadeus (1984)
174.0,Raiders of the Lost Ark (1981)
197.0,"Graduate, The (1967)"
357.0,One Flew Over the Cuckoo's Nest (1975)
483.0,Casablanca (1942)
498.0,"African Queen, The (1951)"
134.0,Citizen Kane (1941)
199.0,"Bridge on the River Kwai, The (1957)"


### Look at the Movies with the Highest/Lowest Model Weights

In [44]:
item_weights = pd.DataFrame({'item_idx': np.arange(len(model.w_item)), 'w_item': model.w_item})
item_weights['item_id'] = item_weights['item_idx'].map(model.index_to_item)
item_weights = pd.merge(item_weights, item_names, on='item_id').sort_values('w_item', ascending=False)
item_weights = item_weights[['item_id', 'item_idx', 'item_name', 'w_item']]

#### Best Movies Ever?

In [45]:
item_weights.head(10)

,item_id,item_idx,item_name,w_item
99,100,99,Fargo (1996),2.575865
49,50,49,Star Wars (1977),2.442906
178,181,178,Return of the Jedi (1983),2.389725
0,1,0,Toy Story (1995),2.229902
55,56,55,Pulp Fiction (1994),2.183959
124,127,124,"Godfather, The (1972)",2.104568
97,98,97,"Silence of the Lambs, The (1991)",2.083257
283,286,283,"English Patient, The (1996)",2.060685
6,7,6,Twelve Monkeys (1995),2.029270
314,318,314,Schindler's List (1993),1.980342


#### Worst Movies Ever?

In [46]:
item_weights.tail(10)

,item_id,item_idx,item_name,w_item
1036,1076,1036,"Pagemaster, The (1994)",-1.734297
1360,1515,1360,Wings of Courage (1995),-1.737375
1268,1380,1268,Hollow Reed (1996),-1.737588
370,375,370,Showgirls (1995),-1.739251
1235,1306,1235,Delta of Venus (1994),-1.740163
1295,1413,1295,Street Fighter (1994),-1.745073
1352,1501,1352,Prisoner of the Mountains (Kavkazsky Plennik) ...,-1.749693
1274,1387,1274,Fall (1997),-1.750252
1229,1299,1229,Penny Serenade (1941),-1.777506
652,666,652,Blood For Dracula (Andy Warhol's Dracula) (1974),-1.782737


# Start Sandbox Code